In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time
from bs4 import BeautifulSoup 
from tqdm import tqdm
import requests
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import re

# 한 셀(cell)에서 하나 이상의 출력을 수행하기 위한 코드
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
# 네이버 지도에서 XX동 맛집을 검색해서 맛집을 클릭하면
# url이 map.naver.ocm/v5/search/연남동%20맛집/place/-----------?c=1412...와 같은 형태임을 볼 수 있었습니다.
# 맛집마다 위 urldml ------자리에 오는 숫자가 다름을 확인하고
# -------자리의 숫자를 크롤링하여 matzip_num이라는 리스트에 넣어두었습니다.
# -------자리에 오는 숫자를 맛집 고유 넘버라고 부르겠습니다.

# 맛집 고유 넘버 넣을 리스트 생성
matzip_num = []

In [8]:
# 검색할 키워드
keyword = '연남동 맛집'
url = f'https://map.naver.com/v5/search/{keyword}/place/'

# chrome driver 설치 후 ()안에 크롬드라이버의 위치를 입력해줍니다.
driver = webdriver.Chrome('/Users/mckimair/chromedriver')
driver.get(url)
time.sleep(2)

# iframe 변경
driver.switch_to.frame('searchIframe')

# 네이버 지도에서 검색한 결과를 보면 6장의 페이지가 나오기 때문에 범위를 range(1,7)로 줍니다
for num in tqdm(range(1,7)): 
    
    # 아래의 코드는 사용자의 컴퓨터 사양에 따라 작업수행속도 등의 차이가 있기 때문에
    # 코드를 더 줄여도 됩니다.
    # driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]').click()
    # driver.find_element_by_tag_name('body').send_keys(Keys.END) # 스크롤 
    # time.sleep(2)
    
    # 스크롤바 클릭
    driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]').click()
    # END 키를 눌러서 페이지 제일 하단으로 이동 = 스크롤 다운
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)
    driver.find_element_by_tag_name('body').send_keys(Keys.END) 
    time.sleep(2)
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(2)

    if num == 1: # 첫번째 페이지에서는 다른 동작을 하지 않고 그대로 크롤링
        pass
    else: # 두번째 페이지 부터는 옆으로 넘어가는 버튼을 클릭하고 크롤링
        driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()



    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
    time.sleep(2)

    matzip_list = soup.findAll('li', {'class':'_1EKsQ _12tNp'})

    for i in tqdm(range(len(matzip_list))):
        driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{str(i+1)}]/div[1]/a').send_keys(Keys.ENTER)
        time.sleep(2)
    
        current_url = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지

        unique_code = re.findall(r"place/(\d+)", current_url)
        matzip_num.append(unique_code[0])
        time.sleep(2)
    
driver.quit()



100%|██████████| 6/6 [22:01<00:00, 220.21s/it]


In [9]:
len(matzip_num)

300

In [10]:
matzip_df = pd.DataFrame()
matzip_df['num'] = matzip_num

In [11]:
matzip_df

,num
0,1485118707
1,1744506906
2,1275815754
3,1753210391
4,662179145
...,...
295,1096273146
296,1124366262
297,1630030343
298,1113794499


In [12]:
# 맛집 고유 넘버를 저장하는 이유
# 크롤링을 하다 보니 크롤링이 중간에 오류가 생기면 처음부터 다시해야하는데
# 다시 고유넘버를 긁어오는 시간을 조금이라도 줄이기 위해 csv파일로 고유넘버를 저장해 놓았습니다.
# 지역 하나만 한다면 다시하는데 시간이 얼마 걸리지 않지만
# 여러 지역을 돌릴 경우에는 시간이 오래 걸리는 문제가 발생해서 저장했습니다.
matzip_df.to_csv('matzip_num.csv')

## 함수화

In [72]:
def crawl_regional_restaurant_num(keyword):
    globals()[f'matzip_num_{keyword}'] = []
    
    url = f'https://map.naver.com/v5/search/{keyword}/place/'

    driver = webdriver.Chrome('/Users/mckimair/chromedriver') # 크롬드라이버 경로 입력
    driver.get(url)
    time.sleep(2)

    # iframe 변경
    driver.switch_to.frame('searchIframe')

    for num in tqdm(range(1,7)): # 네이버 지도에서 맛집을 검색하면 6장의 페이지가 나오기 때문에 6장을 크롤링(범위)

        # 스크롤바 클릭
        driver.find_element_by_xpath('//*[@id="_pcmap_list_scroll_container"]').click()
        # END 키를 눌러서 페이지 제일 하단으로 이동 = 스크롤 다운
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END) 
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        time.sleep(2)

        if num == 1: # 첫번째 페이지에서는 다른 동작을 하지 않고 그대로 크롤링
            pass
        else: # 두번째 페이지 부터는 옆으로 넘어가는 버튼을 클릭하고 크롤링
            driver.find_element_by_xpath('//*[@id="app-root"]/div/div[2]/div[2]/a[7]').click()

        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        time.sleep(2)

        matzip_list = soup.findAll('li', {'class':'_1EKsQ _12tNp'})

        for i in tqdm(range(len(matzip_list))):
            driver.find_element_by_xpath(f'//*[@id="_pcmap_list_scroll_container"]/ul/li[{str(i+1)}]/div[1]/a').send_keys(Keys.ENTER)
            time.sleep(2)

            current_url = driver.current_url # 검색이 성공된 플레이스에 대한 개별 페이지

            unique_code = re.findall(r"place/(\d+)", current_url)
            globals()[f'matzip_num_{keyword}'].append(unique_code[0])
            time.sleep(2)

    driver.quit()
    
    matzip_df = pd.DataFrame()
    matzip_df['num'] = globals()[f'matzip_num_{keyword}']
    matzip_df.to_csv('matzip_num.csv')
    
    return matzip_df


In [74]:
crawl_regional_restaurant_num('고양시 일산동구 맛집')

100%|██████████| 6/6 [22:06<00:00, 221.09s/it]


,num
0,1958395575
1,1591406580
2,38491290
3,1350977248
4,1946278960
...,...
295,1235747016
296,1623633664
297,1161878787
298,1989399123
